In [72]:
!pip install minsearch  groq load_dotenv


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [52]:
import minsearch
import json 

In [53]:
with open('documents.json' , 'rt') as f_in:
    docs_raw = json.load(f_in)
    

In [54]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)


In [55]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [56]:
index = minsearch.Index(
    text_fields=["question" , "text" ,"section"],
    keyword_fields= ["course"]
)

In [10]:
    SELECT * WHERE Course = 'data-engineering-zoomcamp';


SyntaxError: invalid syntax (3333116107.py, line 1)

In [57]:
q = 'the course has already started, can I still enroll?'

In [58]:
index.fit(documents)

In [59]:
boost = {'question': 3.0 , 'section' : 0.5}

results = index.search(
    query = q,
    boost_dict = boost,
    num_results = 5
)

results 

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [60]:
boost = {'question': 3.0 , 'section' : 0.5}

results = index.search(
    query = q,
    boost_dict = boost,
    filter_dict = {'course' : 'data-engineering-zoomcamp'},
    num_results = 5
)

In [61]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [ ]:
#  from openai import OpenAI
# client = OpenAI()

In [75]:
import os 
from groq import Groq 
from dotenv import load_dotenv 
load_dotenv()

client = Groq(api_key = os.environ.get('GROQ_API_KEY'))



In [76]:
response = client.chat.completions.create(
    model= 'llama3-70b-8192' ,
    messages= [{"role": "user", "content":q}]

)

response.choices[0].message.content


"It's great that you're interested in the course!\n\nWhether you can still enroll depends on the course provider and their policies. Here are a few possibilities:\n\n1. **Enrollment deadline**: The course provider might have a specific deadline for joining the course. If you've missed it, you might not be able to enroll.\n2. **Late enrollment**: Some course providers allow late enrollment, but you might need to catch up on the material or complete additional assignments to get up to speed.\n3. **Waitlist**: If the course is full, you might be able to join a waitlist. If a spot becomes available, you'll be offered a place in the course.\n\nTo find out if you can still enroll, I recommend:\n\n* Checking the course website or learning platform for any specific enrollment deadlines or policies.\n* Reaching out to the course provider or instructor directly to ask about late enrollment options.\n* Exploring possible alternative courses or sessions that you can enroll in.\n\nDon't give up! Yo

In [77]:
prompt_template = """ 
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [78]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    

In [79]:
prompt = prompt_template.format(question=q, context=context).strip()

In [80]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fin

In [82]:
response = client.chat.completions.create(
    model= 'llama3-70b-8192',
    messages = [{"role": "user", "content": prompt}]
)

response.choices[0].message.content 

'According to the FAQ, the answer is: YES'

### Modular Code or Clean Code (Functional Programming)

In [83]:
def search(query):
    boost = {'question': 3.0 , 'section' : 0.5}

    results = index.search(
        query = query,
        boost_dict = boost,
        num_results = 5
    )

    return results 

In [84]:
def build_prompt(query, search_results):
    prompt_template = """ 
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt 

In [85]:
def llm(prompt):
    response = client.chat.completions.create(
        model= 'llama3-70b-8192',
        messages= [{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [86]:
query = 'the course already started. Can I still enroll?'


def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [87]:
rag(query)

'According to the CONTEXT, the answer to the QUESTION "Can I still enroll?" is: YES'

In [ ]:
### USE ElasticSearch Build RAG

In [92]:
from elasticsearch import Elasticsearch

In [93]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [94]:
es_client = Elasticsearch(
    ['http://localhost:9200'],
    headers={
        "Accept": "application/vnd.elasticsearch+json; compatible-with=8",
        "Content-Type": "application/json"
    }  
)

In [98]:
print(es_client)

<Elasticsearch(['http://localhost:9200'])>


In [99]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'media_type_header_exception', 'Invalid media-type value on headers [Accept, Content-Type]', Accept version must be either version 8 or 7, but found 9. Accept=application/vnd.elasticsearch+json; compatible-with=9)

In [100]:
from tqdm.auto import tqdm 

In [101]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

BadRequestError: BadRequestError(400, 'media_type_header_exception', 'Invalid media-type value on headers [Accept, Content-Type]', Accept version must be either version 8 or 7, but found 9. Accept=application/vnd.elasticsearch+json; compatible-with=9)

In [102]:
! curl http://localhost:9200 


{
  "name" : "a9c18ccd6336",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "nLSX_6bZRlKEQ5cLf4b4dg",
  "version" : {
    "number" : "8.15.1",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "253e8544a65ad44581194068936f2a5d57c2c051",
    "build_date" : "2024-09-02T22:04:47.310170297Z",
    "build_snapshot" : false,
    "lucene_version" : "9.11.1",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [103]:
query = 'the course already started. Can I still enroll?'

In [104]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index= index_name, body= search_query)

    result_doc = []

    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])

    return result_doc

In [105]:

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

rag(query)

In [106]:
rag(query)

BadRequestError: BadRequestError(400, 'media_type_header_exception', 'Invalid media-type value on headers [Accept, Content-Type]', Accept version must be either version 8 or 7, but found 9. Accept=application/vnd.elasticsearch+json; compatible-with=9)